<center><img src="https://i.imgur.com/10xit0K.png"> </center>

In [ ]:
!pip install autogluon #Restart the kernel after the installation is complete

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

import warnings
warnings.filterwarnings("ignore")

<div style="border-radius:10px; padding: 15px; background-color: #21ace81a; font-size:120%; text-align:left">
      
* Here, we import the features from the training data, performing the necessary type conversions according to how we intend to use them.
* Groupings such as P_Id and S_Id were made as requested in the submission.
* Negative values, which constitute 4.8% of the total QTY amount, were converted to 0, focusing solely on the quantities of products sold.

In [2]:
train = pd.read_csv("/kaggle/input/haier-europe-forecasting-hackathon-with-python/train.csv",
                    parse_dates=["DATE"], 
                    dtype={"AREA_HYERARCHY1": int, "AREA_HYERARCHY2":int},
                    usecols = ["QTY","DATE","AREA_HYERARCHY1","AREA_HYERARCHY2","PRODUCT","PH1"]) \
                    .sort_values(by="DATE").reset_index(drop=True)

sub = pd.read_csv("/kaggle/input/haier-europe-forecasting-hackathon-with-python/sample_submission_joined.csv")

train["P_Id"] = train["PRODUCT"]+","+train["AREA_HYERARCHY1"].astype(str)
train["S_Id"] = train["PH1"]+","+train["AREA_HYERARCHY2"].astype(str)
train.loc[train["QTY"] < 0, "QTY"] = 0

<div style="border-radius:10px; padding: 15px; background-color: #21ace81a; font-size:120%; text-align:left">
      
* Weekly P and Monthly S IDs were exported from the submission file.


In [3]:
pler = sub["Id"].str.extract(r'^(P[^,]*,[^,]*)',expand=False).dropna().drop_duplicates().values
sler = sub["Id"].str.extract(r'^(S[^,]*,[^,]*)',expand=False).dropna().drop_duplicates().values

<div style="border-radius:10px; padding: 15px; background-color: #21ace81a; font-size:120%; text-align:left">
      
* The data with irregular dates in the train data were grouped by product id and week by moving the dates to the start of the weeks as needed, and 'p_data' consisting only of P's with weekly sales was created.
* Weeks without any data entry were recreated and filled in with the interpolation method.

In [4]:
p_data = pd.DataFrame()

p_train = train[(train["P_Id"].isin(pler))][["DATE","P_Id", "QTY"]].copy()
p_train['DATE'] = p_train['DATE'] - pd.to_timedelta(p_train['DATE'].dt.dayofweek, unit='d')
p_train = p_train.groupby(["DATE", "P_Id"]).agg({"QTY": "sum"}).reset_index().rename(columns={"QTY": "target"})
    
progress_bar = tqdm(total=len(pler))
for i in pler:
    data = p_train[(p_train["P_Id"] == i)]
    
    date_range = pd.date_range(start=data['DATE'].min(), end="2023-09-25", freq='W-MON')
    df_all_weeks = pd.DataFrame({'DATE': date_range})
    data = pd.merge(df_all_weeks, data, on='DATE', how='left')
    
    data["P_Id"] , data["target"] = data["P_Id"].ffill(), data["target"].interpolate(method="linear")

    p_data = pd.concat([p_data, data], axis=0, ignore_index=True)
    progress_bar.update(1)

  0%|          | 0/5299 [00:00<?, ?it/s]

<div style="border-radius:10px; padding: 15px; background-color: #21ace81a; font-size:120%; text-align:left">
      
* The data with irregular dates in the train data were grouped by S id and Month by moving the dates to the start of the month as needed, and 's_data' consisting only of S's with monthly sales was created.
* Products that did not have any data last month were removed from the data.
* Months with quantities falling below the 10th percentile, albeit rare, were replaced with NaN and then filled again through interpolation.
* Months with no data input were recreated and assigned as 0.

In [6]:
s_data = pd.DataFrame()

s_train = train[(train["S_Id"].isin(sler))][["DATE", "S_Id", "PRODUCT", "QTY"]]
s_train['DATE'] = s_train['DATE'] - pd.to_timedelta(s_train['DATE'].dt.day - 1, unit='d')

progress_bar = tqdm(total=len(sler))
for i in sler:
    data = s_train[(s_train["S_Id"] == i)]
    
    churn_items = data.groupby(["PRODUCT"]).agg({"DATE": "last"})
    churn_items_indexes = churn_items[churn_items["DATE"] < "2023-09-01"].index
    data = data[~data["PRODUCT"].isin(churn_items_indexes)]

    data = data.groupby(["DATE", "S_Id"]).agg({"QTY": "sum"}).reset_index().rename(columns={"QTY": "target"})
    
    data.loc[data["target"] < data["target"].quantile(0.1), "target"] = np.NaN
    data["target"] = data["target"].interpolate(method="linear")
    
    if data.empty == False:
        date_range = pd.date_range(start=data['DATE'].min(), end="2023-09-01", freq='MS')
        df_all_months = pd.DataFrame({'DATE': date_range})
        data = pd.merge(df_all_months, data, on='DATE', how='left')
        
        data["S_Id"], data["target"] = data["S_Id"].ffill(), data["target"].fillna(0)
        
        s_data = pd.concat([s_data, data], axis=0, ignore_index=True)
    else:
        s_data = pd.concat([s_data, data], axis=0, ignore_index=True)
    progress_bar.update(1)

  0%|          | 0/1259 [00:00<?, ?it/s]

<div style="border-radius:10px; padding: 15px; background-color: #21ace81a; font-size:120%; text-align:left">
      
* The data was converted to the data format that Autogluon expected from us with the TimeSeriesDataFrame method, and the time interval to be forecast and the frequency of the data were specified.
* The number of weeks was added to the IDs as the competition expected from us.

In [ ]:
autogluon_p = TimeSeriesDataFrame.from_data_frame(
    p_data,
    id_column="P_Id",
    timestamp_column="DATE",
)

p_predics = TimeSeriesPredictor(target='target',
                                prediction_length=12,
                                eval_metric="RMSE",
                                freq="W-MON").fit(autogluon_p).predict(autogluon_p).reset_index()

p_predics["timestamp"] = p_predics["item_id"].nunique() * [1,2,3,4,5,6,7,8,9,10,11,12]

p_predics["Id"] = [f"{i},{t}" for i,t in p_predics[["item_id","timestamp"]].values]

p_results = p_predics[["Id","mean"]].rename(columns={"mean":"QTY"})

<div style="border-radius:10px; padding: 15px; background-color: #21ace81a; font-size:120%; text-align:left">
      
* The data was converted to the data format that Autogluon expected from us with the TimeSeriesDataFrame method, and the time interval to be forecast and the frequency of the data were specified.
* The number of months was added to the IDs as the competition expected from us.

In [ ]:
autogluon_s = TimeSeriesDataFrame.from_data_frame(
    s_data,
    id_column="S_Id",
    timestamp_column="DATE",
)


s_predics = TimeSeriesPredictor(target='target',
                                prediction_length=3,
                                eval_metric="RMSE",
                                freq="MS").fit(autogluon_s).predict(autogluon_s).reset_index()

s_predics["timestamp"] = s_predics["item_id"].nunique() * [1,2,3]

s_predics["Id"] = [f"{i},{t}" for i,t in s_predics[["item_id","timestamp"]].values]

s_results = s_predics[["Id","mean"]].rename(columns={"mean":"QTY"})

<div style="border-radius:10px; padding: 15px; background-color: #21ace81a; font-size:120%; text-align:left">
      
* Values that did not contain any values were assigned as 0, assuming that they would not occur in the next 3 months. 
* Since small amounts of negative values were not in our prediction target, they were assigned as 0 and a submission file was created.

In [30]:
total_results = pd.concat([p_results,s_results],ignore_index=True)
endsub = sub[["Id"]].merge(total_results,on=["Id"], how="left")

endsub["QTY"] = endsub["QTY"].fillna(0)
endsub.loc[endsub["QTY"] < 0,"QTY"] = 0
endsub["QTY"] = endsub["QTY"].round()

In [32]:
endsub.to_csv("submission.csv", index=False)